In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import Lasso

# Carregamento dos dados 

In [2]:
df1 = pd.read_csv("data/sample/sampleAccepted.csv")
df1 = df1.drop(columns=["member_id", "url"])
df1.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 149 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   id                                          10000 non-null  int64  
 1   loan_amnt                                   10000 non-null  float64
 2   funded_amnt                                 10000 non-null  float64
 3   funded_amnt_inv                             10000 non-null  float64
 4   term                                        10000 non-null  object 
 5   int_rate                                    10000 non-null  float64
 6   installment                                 10000 non-null  float64
 7   grade                                       10000 non-null  object 
 8   sub_grade                                   10000 non-null  object 
 9   emp_title                                   9257 non-null   object 
 10  emp_length

# Seleção das variáveis

Não serão usadas todas as variáveis disponíveis.

Manualmente selecionamos algumas variáveis que consideramos mais relevantes.

In [3]:
features = df1.loc[:, [
    "term", # Número de pagamentos pagamentos no plano
    "annual_inc", # Renda Anual
    "annual_inc_joint", # Renda anual conjunta se o empréstimo é conjunto
    "emp_length", # Tempo no emprego
    "issue_d", # data do empréstimo, para comparar com a taxa de juros básica
    "purpose", # objetivo
    "dti", # Debt-to-income <https://www.investopedia.com/terms/d/dti.asp>
    "delinq_2yrs", # Número de crimes nos últimos 2 anos
    "earliest_cr_line", # data de abertura da última linha de crédito
    "installment", # Prestação
    "open_acc", # Número de linhas de crédito abertas no histórico do cliente
    "open_il_12m", # Número de contas de prestação abertas nos últimos 12m
    "open_rv_12m", # Número de contas de crédtio rotativo abertas nos últimos 12m
    "pub_rec", # Number of derogatory public records
    "revol_util", # Fração do crédito giratório disponível sendo usado
    "application_type", # Se a aplicação é individual ou conjunta
    "chargeoff_within_12_mths", # número de declarações de 'improvável pagar' nos últimos 12m
    "mort_acc", # número de hipotecas
    "num_accts_ever_120_pd", # número de contas que já foram atrasadas a mais de 120 dias
    "num_sats", # Número de contas 'satisfatórias'
    "num_tl_120dpd_2m", # número de contas atualmente atrasadas mais de 120 dias(atualizadas nos últimos 2 meses)
    "num_tl_30dpd", # Número de contas atualemente atrasadas mais de 30 dias
    "pct_tl_nvr_dlq", # fração de transações não 'delinquency'
    "percent_bc_gt_75", # fração de contas acima de 75% do limite
    "tax_liens", # reinvindicação do governo em razão de taxas atrasadas
    "hardship_flag", # se está em plano de pagamento especial do cartão
    "disbursement_method", # Metodo de recebimento do pagamento
    ]]

features.head()

,term,annual_inc,annual_inc_joint,emp_length,issue_d,purpose,dti,delinq_2yrs,earliest_cr_line,installment,...,mort_acc,num_accts_ever_120_pd,num_sats,num_tl_120dpd_2m,num_tl_30dpd,pct_tl_nvr_dlq,percent_bc_gt_75,tax_liens,hardship_flag,disbursement_method
0,36 months,62000.0,NaN,3 years,Dec-2018,other,13.38,0.0,Nov-2003,368.15,...,1.0,0.0,8.0,0.0,0.0,95.5,0.0,0.0,N,Cash
1,36 months,69000.0,NaN,8 years,Mar-2017,debt_consolidation,30.90,0.0,Feb-2007,285.19,...,0.0,1.0,12.0,0.0,0.0,77.3,33.3,0.0,N,Cash
2,36 months,102000.0,NaN,2 years,Jul-2018,credit_card,18.49,0.0,Oct-1997,784.34,...,0.0,0.0,12.0,0.0,0.0,100.0,33.3,0.0,N,DirectPay
3,36 months,33000.0,NaN,10+ years,Sep-2015,debt_consolidation,24.62,0.0,Aug-1988,234.65,...,6.0,0.0,7.0,0.0,0.0,100.0,0.0,0.0,N,Cash
4,36 months,32201.0,NaN,10+ years,Mar-2016,debt_consolidation,28.14,0.0,Jan-2001,378.20,...,0.0,0.0,11.0,0.0,0.0,100.0,0.0,0.0,N,Cash


# Limpeza dos dados

Diversos dados estão em formatos não apropriados ou devem ser transformados em outras variáveis.

Por motivos de performance e legibilidade geralmente será criada uma função e usada o método apply do pandas.

## Formatando *Term*

A coluna Term veio em formato de texto, porém é mais apropriado que seja um número inteiro.

In [4]:
def formatTerm(line):
    line = line.strip()
    line = line.split(" ")

    if line[-1] == "months":
        line = int(line[0])

    else:
        raise NotImplementedError(f"{line[-1]} não implementado ainda")

    return line

features["term"] = features["term"].apply(formatTerm)

## Dummyficação de OWNERSHIP

A coluna OWNERSHIP será transformada em dummies.

O valor OWN será descartado para evitar multicolinearidade perfeita.

In [5]:
features = features.join(pd.get_dummies(df1["home_ownership"]))
features.drop(columns="OWN")

ownership = ["ANY", "MORTGAGE", "NONE", "OTHER", "RENT"]

## Adaptação de *EmpLength*

Da coluna *EmpLength* queremos saber apenas se o indivíduo está no trabalho a mais de um ano.

In [6]:
def formatEmpLength(line):
    """ Testa se o indivíduo está a mais de dois anos no trabalho atual """
    Accept = ["3 years", "2 years", "8 years", "10+ years", "4 years", "6 years", "5 years", "9 years", "7 years"]
    NotAccept = ["1 year", "< 1 year", np.nan]

    if line in Accept:
        return True
    elif line in NotAccept:
        return False
    else:
        raise NotImplementedError(f"{line} não implementado ainda na função cleanEmpLength")

features["emp_length"] = features["emp_length"].apply(formatEmpLength)


## Dummyficação de *purpose*

A coluna será transformada em três outras variáveis:

1. GoodPurpose: se o uso original é: 'small_business', 'renewable_energy', 'home_improvement', 'major_purchase', 'moving', 'house' ou 'car'

2. BadPurpose: se o uso original é: 'debt_consolidation', 'credit_card' ou 'medical'

3. NeutralPurpose: se o uso original é: 'other', 'vacation' ou 'wedding'

In [7]:
def toIntPurpose(line):
    goodList = ["small_business", "renewable_energy", "home_improvement", "major_purchase", "moving", "house", "car"]
    badList = ["debt_consolidation", "credit_card", "medical"]
    neutralList = ["other", "vacation", "wedding"]

    if line in goodList:
        return 1

    elif line in badList:
        return -1

    elif line in neutralList:
        return 0

    else:
        raise NotImplementedError(f"Purpose {line} não está em nenhuma categoria")

features = features.join(
    pd.get_dummies(
    features["purpose"]
    .apply(toIntPurpose))
    .rename(columns={
        -1: "BadPurpose",
        0: "NeutralPurpose",
        1: "GoodPurpose"
        })
    )

features = features.drop(columns="purpose")

In [8]:
target = df1[["int_rate", "grade", "loan_status"]]
target.head()

,int_rate,grade,loan_status
0,26.31,E,Late (31-120 days)
1,16.99,D,Charged Off
2,8.08,A,Current
3,7.89,A,Fully Paid
4,8.39,B,Fully Paid
